In [1]:
import pandas as pd
from datetime import datetime
import os.path
from dotenv import load_dotenv
import awswrangler as wr
import h3

In [2]:
load_dotenv()
DATA_DIR = os.environ.get("DATA_DIR")
S3_DATA_DIR = os.environ.get("S3_DATA_DIR")

## Using Pandas to Read Parquet Files

In [4]:
data_pd = pd.read_parquet(f"{DATA_DIR}\df.parquet.gzip")
print(data_pd.count()) # example of operation on the returned DataFrame

time1_bounds                               4152960
precipitation_amount_1hour_Accumulation    4152960
dtype: int64


## Reset Index to Datetime

In [ ]:
start_time = pd.to_datetime("2022-05-01 21:00:00") #inclusive
end_time = pd.to_datetime("2022-05-01 23:00:00") #not inclusive

In [ ]:
# Reset index so the time1 is only index
data_pd_time_index = data_pd.reset_index(level=['lon','lat','nv'])

In [ ]:
# Set index to pandas datetime
data_pd_time_index.index = pd.to_datetime(data_pd_time_index.index)

In [ ]:
# Filter by time
data_pd_time_index.loc[(data_pd_time_index.index>=start_time)&(data_pd_time_index.index<end_time)]

## Filtering by the hierarchical geospatial index

In [ ]:
# Add h3 field to the dataframe
data_pd_time_index['h3_index'] = data_pd_time_index.apply(lambda row: h3.geo_to_h3(row['lat'], row['lon'], 2), axis=1)

In [ ]:
# Test a query using h3
h3_index = '820327fffffffff'
filtered_df = data_pd_time_index[data_pd_time_index['h3_index'] == h3_index]

In [ ]:
filtered_df.describe()

## Exploring AWS Wrangler for Reading Parquet Files from S3

In [6]:
generator = wr.s3.read_parquet(path = f"{S3_DATA_DIR}/precipitation_data/", chunked=1_000_000)

NameError: name 'S3_Data_DIR' is not defined

In [5]:
# Loop through the first generator created to reset indices
for df in generator:
    df_time_index = df.reset_index(level=['lon','lat','nv'])
    pd.to_datetime(df_time_index.index)
    print(df_time_index.index.day)
    break
    

NameError: name 'generator' is not defined